# 1. Frame the Problem
Create a model that can be used in order to play a game of tetris for as long as possible.

# 2. Get the Data
We have been provided with the already coded game of tetris which will be used to create a model.

In [1]:
import numpy as np
import pandas as pd
import operator
from copy import copy, deepcopy
import random
from piece import BODIES, Piece
from board import Board
from myalgo import myai
from game import Game
from genetic_controller import compute_fitness

pygame 2.5.2 (SDL 2.28.2, Python 3.10.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
population = 50
gens = 10
trials = 5
num_elite = 5
survival_rate=.2
logging_file='training5.csv'
headers = ['avg_fit','avg_gene', 'top_fit', 'top_gene', 'elite_fit', 'elite_gene']
pop = [myai(num_features = 3) for x in range(population)]

In [3]:
def playtetris(ai):
    game = Game(mode ='myai')
    pieces, cleared = game.run_no_visual()
    return pieces

In [4]:
def cross(a1, a2):
    new_genotype = []
    a1_prop = a1.fit_rel / a2.fit_rel
    for i in range(len(a1.genotype)):
        rand = random.uniform(0, 1)
        if rand > a1_prop:
            new_genotype.append(a1.genotype[i])
        else:
            new_genotype.append(a2.genotype[i])

    return myai(genotype=np.array(new_genotype), aggregate='lin', mutate=True)

In [5]:
for gen in range(gens):

        total_fitness = 0
        top_agent = 0
        gene = np.zeros(3)

        for n in range(population):
            print(f"Agent: {n}/{population}")
            agent = pop[n]
            agent.fit_score = compute_fitness(agent, num_trials=trials)
            total_fitness += agent.fit_score 
            #try:
            print(agent.genotype)
            print(gene)
            gene+=agent.genotype


        for agent in pop:
            agent.fit_rel = agent.fit_score / total_fitness

        next_gen = []

        sorted_pop = sorted(pop, reverse=True)

        elite_fit_score = 0
        elite_genes = np.zeros(3)
        top_agent=sorted_pop[0]

        for i in range(num_elite):
            elite_fit_score +=sorted_pop[i].fit_score
            elite_genes += sorted_pop[i].genotype
            next_gen.append(myai(genotype=sorted_pop[i].genotype, mutate=False))

       
        num_parents = round(population * survival_rate)
        parents = sorted_pop[:num_parents]

        
        for _ in range(population-(num_elite)):
            
            parents = random.sample(parents, 2)
            next_gen.append(cross(parents[0], parents[1]))


        avg_fit = (total_fitness/population)
        avg_gene = (gene/population)
        top_fit = (top_agent.fit_score)
        top_gene = (top_agent.genotype)
        elite_fit = (elite_fit_score/num_elite)
        elite_gene = (elite_genes/num_elite)

        data = [[avg_fit, avg_gene, top_fit, top_gene, elite_fit, elite_gene]]
        df = pd.DataFrame(data, columns=headers)
        df.to_csv(f'data/{logging_file}.csv', mode='a', index=False, header=False)


        print(f'\nEpoch {gens}: \n    total fitness: {total_fitness/population}\n    best agent: {top_agent.fit_score}\n')

        pop = next_gen

Agent: 0/50
20 0
    Trial: 0/5
27 3
    Trial: 1/5
16 1
    Trial: 2/5
13 0
    Trial: 3/5
17 1
    Trial: 4/5
[0.65185025 0.13331266 0.78404375]
[0. 0. 0.]
Agent: 1/50
22 3
    Trial: 0/5
15 1
    Trial: 1/5
9 0
    Trial: 2/5
16 1
    Trial: 3/5
14 1
    Trial: 4/5
[-0.70908544  0.36420843  0.75969323]
[0.65185025 0.13331266 0.78404375]
Agent: 2/50
22 1
    Trial: 0/5
18 1
    Trial: 1/5
39 3
    Trial: 2/5
19 2
    Trial: 3/5
35 2
    Trial: 4/5
[0.42735804 0.28079777 0.57229751]
[-0.05723519  0.49752109  1.54373698]
Agent: 3/50
195 63
    Trial: 0/5
150 43
    Trial: 1/5
100 23
    Trial: 2/5
101 23
    Trial: 3/5
265 92
    Trial: 4/5
[ 0.42251403  0.49191529 -0.27577646]
[0.37012285 0.77831887 2.11603449]
Agent: 4/50
10 0
    Trial: 0/5
34 8
    Trial: 1/5
17 0
    Trial: 2/5
28 2
    Trial: 3/5
23 0
    Trial: 4/5
[-0.55279975 -0.49106954  0.13839323]
[0.79263688 1.27023416 1.84025803]
Agent: 5/50
18 2
    Trial: 0/5
20 2
    Trial: 1/5
29 4
    Trial: 2/5
18 1
    Trial: 3/5
2

In [26]:
%run main.py myai

217 73
